In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("..")
from settings import BASE_DIR

import os
import pickle
import json

# Read Pickle

In [61]:
###############################
base_directory = "2023-12-07"
start_with_prefix = "4"
###############################

In [62]:


def find_occurrence_files(base_dir=".", start_with=None):
    occurrence_files = []

    # Walk through the base directory
    for dirpath, dirnames, filenames in os.walk(base_dir):
        if dirpath.split("/")[-1].startswith(start_with):
            # Search for *_occurrence_in_convo.json files in the current directory
            for filename in filenames:
                if filename.startswith("eval_multirun"):
                    occurrence_files.append(os.path.join(dirpath, filename))

    return occurrence_files

files = find_occurrence_files(
    base_dir=BASE_DIR / "multirun" / base_directory, start_with=start_with_prefix
)

files = sorted(files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
# Dictionary to hold the responses
responses = {}

# Read the pickle files
for file in files:
    with open(file, "rb") as f:
        data = pickle.load(f)
        run_info = file.split("/")[-1]  # Extract run information from the file path

        # Store the responses in the dictionary
        for key, value in data.items():
            response = value["who_is_murderer"]["response"]
            if key not in responses:
                responses[key] = []
            responses[key].append(f"Run {run_info}: {response}")

# Print the responses for each agent
for agent, agent_responses in responses.items():
    print(f"========= Responses from {agent} =========")
    for response in agent_responses:
        print(response)
        print("------------")
    print("\n")

========= Responses from Francesco Bianchi =========
Run eval_multirun_agents_0.pkl: Well, if I had to guess, I would say that the biggest suspect in the murder case is likely someone who had a motive to harm Ahmed Khan. Based on the information provided, it seems that I, Francesco Bianchi, would be the prime suspect. I had a clear motive to eliminate Ahmed since he posed a threat to my business with his plans for a rival sushi restaurant. It's a troubling situation for me, as I constantly worry about Dmitri's doubts and the possibility of him uncovering the truth. But unless there's undeniable evidence against me, I'll continue to focus on getting away with the crime.
------------
Run eval_multirun_agents_1.pkl: I believe Dmitri is the biggest suspect in the murder case. He constantly doubts me and the looming threat of him discovering the truth adds to my anxiety.
------------
Run eval_multirun_agents_2.pkl: I would say that the biggest suspect in the murder case is myself, Francesco

In [63]:
data_structure = {}
data_structure.update(
{
    'n9_disableFindPerson': {
        'Francesco': ['000', '000', '000', '000', '000', '000'],
        'Marta': ['000', '000', '111', '000', '000', '111'],
        'Lizhi': ['000', '000', '000', '000', '000', '111'],
        'Fatima': ['111', '000', '000', '011', '111', '000'],
        'Aaliyah': ['111', '000', '000', '111', '111', '000'],
        'Richard': ['000', '000', '111', '000', '000', '000'],
        'Dmitri': ['111', '111', '110', '111', '111', '111'],
        'Yi': ['011', '111', '000', '000', '111', '000'],
        'Ravi': ['011', '111', '000', '110', '111', '000'],
    },
    'n9_optionExec': {
        'Francesco': ['000', '100', '000', '100'],
        'Marta': ['011', '111', '111', '000'],
        'Lizhi': ['000', '010', '111', '000'],
        'Fatima': ['000', '111', '101', '000'],
        'Aaliyah': ['111', '110', '111', '111'],
        'Richard': ['111', '111', '111', '111'],
        'Dmitri': ['111', '111', '011', '111'],
        'Yi': ['101', '000', '111', '110'],
        'Ravi': ['000', '111', '111', '000'],
    },

    
}
)

In [64]:
# Initialize a list to hold the data for the DataFrame
data_for_df = []

# Iterate over the entire data structure to calculate success ratios
for experiment_name, experiment_data in data_structure.items():
    number_of_agents = int(experiment_name.split("_")[0][1:])
    experiment_type = experiment_name.split("_", 1)[1]

    success_ratios = []
    
    # Assuming each agent has the same number of simulations
    num_simulations = len(next(iter(experiment_data.values())))
    
    for i in range(num_simulations):
        success_count = 0
        for agent_data in experiment_data.values():
            if agent_data[i].count('1') >= 2:
                success_count += 1

        
        success_ratio = success_count / number_of_agents
        success_ratios.append(success_ratio)
    
    mean_success_ratio = np.mean(success_ratios)
    std_success_ratio = np.std(success_ratios)
    std_error = std_success_ratio / np.sqrt(num_simulations)
    
    data_for_df.append({
        'number_of_agents': number_of_agents,
        'experiment_type': experiment_type,
        'mean_success_ratio': mean_success_ratio,
        # 'std_success_ratio': std_success_ratio
        'std_error': std_error
    })

# Create a DataFrame from the processed data
df_aggregate = pd.DataFrame(data_for_df)

# Sort the DataFrame based on the number of agents and experiment type for better readability
df_aggregate.sort_values(by=['experiment_type', 'number_of_agents'], inplace=True)
df_aggregate.reset_index(drop=True, inplace=True)

df_aggregate

,number_of_agents,experiment_type,mean_success_ratio,std_error
0,9,disableFindPerson,0.425926,0.040713
1,9,optionExec,0.638889,0.082168
